In [3]:
#imports

import scipy
import numpy as np
import pandas as pd
#from fforma import FFORMA
import pickle
from functools import partial
import multiprocessing as mp
import glob
#from tsfeatures import tsfeatures
from os import listdir
from os.path import isfile, join

In [4]:
#run fforma scripts.
%run './Extra_packages/fforma/fforma.py'

#run tsfeatures scripts.
%run './Extra_packages/tsfeatures/tsfeatures.py'

In [5]:
"""
There are a few booleans and variables included determining which parts are run:
"""

#Boolean that determines whether time series should be reforecasted
reload_forec = False 
reload_feats = False
reload_CI = False
reload_meta_learn = False

#Variable that determines how long the forecasting horizon is
horizon=12 

# Load initial data

In [6]:
#Change file to correct folder.
original_train = pd.read_csv('./Datafiles/Monthly-train.csv')
original_test = pd.read_csv('./Datafiles/Monthly-test.csv')

In [7]:
original_train_T = original_train.set_index('V1')
original_test_T = original_test.set_index('V1')

# Produce forecasts

In [8]:
#Helper file that has all forecasting models
%run models.py

In [9]:
"""
This cell produces and saves all time series forecasts used to train and 
test the FFORMA model. Running this cell for 8000 time series takes around 48 hours
to complete. Note, there is a boolean called CI, which determines in the trainBasicModels function whether it returns point forecasts
or prediction intervals. Do note, that not all models in the model pool can produce prediction intervals.
"""

#Models that are used in the model pool
basic_models = {'Naive': Naive(), 'Naive2':Naive2(), 'SNaive': SeasonalNaive(), 'RWdrift': RandomWalkDrift(), 'ETS':ETS(),
               'AutoArima': AutoArima(), 'TBATS':TBATSFF(), 'DTrendS':DTRENDS(), 'RWDriftS':RWDriftS(), 'LLevelS': LLevelS(), 'LLDTrends':LLDTRENDS()}


if reload_forec:

    #Get forecasts for the first 8000 time series in the TRAINING set.
    for x in range(80):

        #Create list that you want to forecast
        original_train_subs = original_train.iloc[100*x : 100*x + 100]
        #original_train_subs = original_train.iloc[x : x +1]
        original_train_subs = original_train_subs.set_index('V1')
        original_train_subs = original_train_subs.T
        unique_id_lst = original_train_subs.columns.tolist()

        #Create time series and let it train :-12 elements.
        ts_val_train = []
        for i in original_train_subs.columns:
            ts_val_train.append(original_train_subs[i].dropna().tolist()[:-horizon])

        #Create results df
        new_index = pd.MultiIndex.from_tuples(zip(np.repeat(unique_id_lst, horizon), list(range(1, horizon+1))*100), names = ['unique_id', 'ds'])

        preds_df_new = pd.DataFrame(index = new_index) 

        #Create predictions
        ts_val_train_preds = trainBasicModels().train(basic_models, ts_val_train, frcy).predict(horizon, CI = False) 

        #Put predictions in dataframe
        k=0 
        for i in list(basic_models.keys()):
            pred_tot = []
            for j in ts_val_train_preds:
                pred_tot += list(j[k])
            preds_df_new[i]  = pred_tot
            k += 1

        preds_df_new.to_pickle('./Results/Results_curr/Preds_set_{}_{}.pkl'.format(x, horizon))


    #Get forecasts for the first 8000 time series in the TESTING set.
    for x in range(80):

        #Create list that you want to forecast
        original_train_subs = original_train.iloc[100*x : 100*x + 100]
        original_train_subs = original_train_subs.set_index('V1')
        original_train_subs = original_train_subs.T
        unique_id_lst = original_train_subs.columns.tolist()

        #Create time series and let it train all elements.
        ts_val_train = []
        for i in original_train_subs.columns:
            if horizon == 12:
                ts_val_train.append(original_train_subs[i].dropna().tolist())
            elif horizon == 36:
                ts_val_train.append(original_train_subs[i].dropna().tolist()[:-horizon+12])

        #Create results df
        new_index = pd.MultiIndex.from_tuples(zip(np.repeat(unique_id_lst, horizon), list(range(1, horizon+1))*100), names = ['unique_id', 'ds'])

        preds_df_new = pd.DataFrame(index = new_index) 

        #Create predictions
        ts_val_train_preds = trainBasicModels().train(basic_models, ts_val_train, frcy).predict(horizon, CI = False) #takes 1min30

        #Put predictions in dataframe
        k=0 
        for i in list(basic_models.keys()):
            pred_tot = []
            for j in ts_val_train_preds:
                pred_tot += list(j[k])
            preds_df_new[i]  = pred_tot
            k += 1

        preds_df_new.to_pickle('./Results/Results_curr/Preds_set_{}_full_{}.pkl'.format(x, horizon))

# Load Forecasts and do initial manipulation

In [10]:
"""This cell loads all forecasts and combines them into one file"""
if reload_forec:
    new_preds = pd.DataFrame()
    for i in range(80):
        new_preds = new_preds.append(pd.read_pickle('./Results/Results_curr/Preds_set_{}_{}.pkl'.format(i, horizon)))
    new_preds.to_pickle('./Results/Preds_df_train_{}.pkl'.format(horizon))

    #Work on adding new pkl files
    new_preds = pd.DataFrame()
    for i in range(80):
        new_preds = new_preds.append(pd.read_pickle('./Results/Results_curr/Preds_set_{}_full_{}.pkl'.format(i, horizon)))
    new_preds.to_pickle('./Results/Preds_df_test_{}.pkl'.format(horizon))


In [11]:
CI_df_train_12 = pd.read_pickle('./Results/CI_df_train_12.pkl')
CI_df_test_12 = pd.read_pickle('./Results/CI_df_test_12.pkl')
preds_df_tot_train_12 = pd.read_pickle('./Results/Preds_df_train_12.pkl')
preds_df_tot_test_12 = pd.read_pickle('./Results/Preds_df_test_12.pkl')
preds_df_tot_train_36 = pd.read_pickle('./Results/Preds_df_train_36.pkl')
preds_df_tot_test_36 = pd.read_pickle('./Results/Preds_df_test_36.pkl')


In [12]:
#Change indices
CI_df_train_12.index = pd.MultiIndex.from_tuples(zip(np.repeat(range(1, 8001), 12), list(range(1, 13))*8000), names=['unique_id', 'ds']) 
CI_df_test_12.index = pd.MultiIndex.from_tuples(zip(np.repeat(range(1, 8001), 12), list(range(1, 13))*8000), names=['unique_id', 'ds']) 

#Keep only the first 8000 time series
preds_df_tot_train_CI_12 = preds_df_tot_train_12.loc[preds_df_tot_train_12.index.get_level_values(0).isin(range(8001))]
preds_df_tot_test_CI_12 = preds_df_tot_test_12.loc[preds_df_tot_test_12.index.get_level_values(0).isin(range(8001))]
                                                  
                                                  

In [13]:
#Get the max index
max_nr_12 = list(set(list(preds_df_tot_train_12.index.get_level_values(0))))
max_nr_12 = np.max(max_nr_12)

max_nr_36 = list(set(list(preds_df_tot_train_36.index.get_level_values(0))))
max_nr_36 = np.max(max_nr_36)

In [14]:
#Get the original time series, and drop the abundant time series in the datafile
original_unique_id_12 = ['M' + str(x) for x in range(1, max_nr_12+1)]
original_train_T_12 = original_train_T.iloc[:len(original_unique_id_12)]
original_train_T_12 = original_train_T_12.T

original_test_T_12 = original_test_T.iloc[:len(original_unique_id_12)]
original_test_T_12 = original_test_T_12.T

original_unique_id_36 = ['M' + str(x) for x in range(1, max_nr_36+1)]
original_train_T_36 = original_train_T.iloc[:len(original_unique_id_36)]
original_train_T_36 = original_train_T_36.T

original_test_T_36 = original_test_T.iloc[:len(original_unique_id_36)]
original_test_T_36 = original_test_T_36.T

In [15]:
if reload_feats:
    
    #horizon 12, train
    ts_val_train = []


    for i in original_unique_id_12[:8000]:
        #use last three years of testing set
        ts_val_train.append(original_train_T_12[i].dropna().tolist()[-48:-12])

    train_feats_new = pd.DataFrame()

    testje_fin = pd.DataFrame(pd.DataFrame(ts_val_train).stack())
    testje_fin = testje_fin.rename_axis(index = ['unique_id', 'ds'])
    testje_fin = testje_fin.rename(columns = {0:'y'})

    constant_indices = pd.DataFrame(testje_fin.groupby('unique_id')['y'].std())
    constant_indices = constant_indices.loc[constant_indices.y == 0]
    constant_indices = list(constant_indices.index)

    testje_fin = testje_fin.loc[~testje_fin.index.get_level_values(0).isin(constant_indices)]

    testje_new = tsfeatures(testje_fin, 12)
    testje_new.to_pickle('./Results/Feats_train_12_3yr.pkl')
    

    #horizon 12, test
    ts_val_train = []


    for i in original_unique_id_12[:8000]:
        #use last three years of testing set
        ts_val_train.append(original_train_T_12[i].dropna().tolist()[-36:])

    train_feats_new = pd.DataFrame()

    testje_fin = pd.DataFrame(pd.DataFrame(ts_val_train).stack())
    testje_fin = testje_fin.rename_axis(index = ['unique_id', 'ds'])
    testje_fin = testje_fin.rename(columns = {0:'y'})

    constant_indices = pd.DataFrame(testje_fin.groupby('unique_id')['y'].std())
    constant_indices = constant_indices.loc[constant_indices.y == 0]
    constant_indices = list(constant_indices.index)

    testje_fin = testje_fin.loc[~testje_fin.index.get_level_values(0).isin(constant_indices)]

    testje_new = tsfeatures(testje_fin, 12)
    testje_new.to_pickle('./Results/Feats_test_12_3yr.pkl')    

    #horizon 36, train    
    ts_val_train = []    
    
    for i in original_unique_id_36[:8000]:
        ts_val_train.append(original_train_T_36[i].dropna().tolist()[-72:-36])

    train_feats_new = pd.DataFrame()

    testje_fin = pd.DataFrame(pd.DataFrame(ts_val_train).stack())
    testje_fin = testje_fin.rename_axis(index = ['unique_id', 'ds'])
    testje_fin = testje_fin.rename(columns = {0:'y'})

    constant_indices = pd.DataFrame(testje_fin.groupby('unique_id')['y'].std())
    constant_indices = constant_indices.loc[constant_indices.y == 0]
    constant_indices = list(constant_indices.index)

    testje_fin = testje_fin.loc[~testje_fin.index.get_level_values(0).isin(constant_indices)]

    testje_new = tsfeatures(testje_fin, 12)
    testje_new.to_pickle('./Results/Feats_train_36_3yr.pkl')
 
    #horizon 36, test    
    ts_val_train = []    
    
    for i in original_unique_id_36[:8000]:
        ts_val_train.append(original_train_T_36[i].dropna().tolist()[-72+12:-36+12])

    train_feats_new = pd.DataFrame()

    testje_fin = pd.DataFrame(pd.DataFrame(ts_val_train).stack())
    testje_fin = testje_fin.rename_axis(index = ['unique_id', 'ds'])
    testje_fin = testje_fin.rename(columns = {0:'y'})

    constant_indices = pd.DataFrame(testje_fin.groupby('unique_id')['y'].std())
    constant_indices = constant_indices.loc[constant_indices.y == 0]
    constant_indices = list(constant_indices.index)

    testje_fin = testje_fin.loc[~testje_fin.index.get_level_values(0).isin(constant_indices)]

    testje_new = tsfeatures(testje_fin, 12)
    testje_new.to_pickle('./Results/Feats_test_36_3yr.pkl')    
    
    
    
    

In [16]:
#Order of feats that has been used in the training of the meta-learning models per horizon

columns_12 = ['hw_beta', 'unitroot_pp', 'alpha', 'arch_lm', 'flat_spots',
       'diff1_acf10', 'curvature', 'beta', 'garch_acf', 'seas_pacf',
       'seasonal_strength', 'hw_gamma', 'seas_acf1', 'x_pacf5', 'spike',
       'seasonal_period', 'peak', 'hurst', 'unitroot_kpss', 'trend', 'x_acf1',
       'diff2_acf1', 'x_acf10', 'entropy', 'diff2_acf10', 'e_acf10',
       'stability', 'garch_r2', 'trough', 'diff2x_pacf5', 'lumpiness',
       'diff1x_pacf5', 'arch_r2', 'nonlinearity', 'nperiods', 'diff1_acf1',
       'hw_alpha', 'arch_acf', 'linearity', 'e_acf1', 'series_length',
       'crossing_points']

columns_36 = ['alpha', 'arch_acf', 'arch_lm', 'arch_r2', 'beta', 'crossing_points',
       'curvature', 'diff1_acf1', 'diff1_acf10', 'diff1x_pacf5', 'diff2_acf1',
       'diff2_acf10', 'diff2x_pacf5', 'e_acf1', 'e_acf10', 'entropy',
       'flat_spots', 'garch_acf', 'garch_r2', 'hurst', 'hw_alpha', 'hw_beta',
       'hw_gamma', 'linearity', 'lumpiness', 'nonlinearity', 'nperiods',
       'peak', 'seas_acf1', 'seas_pacf', 'seasonal_period',
       'seasonal_strength', 'series_length', 'spike', 'stability', 'trend',
       'trough', 'unitroot_kpss', 'unitroot_pp', 'x_acf1', 'x_acf10',
       'x_pacf5']

In [17]:
"""
Load the features used for training and testing.

XXXX: Check whether this is the correct set of features (see files)
"""

train_feats_tot_12 = pd.read_pickle('./Results/Feats_train_12_3yr.pkl')
test_feats_tot_12 = pd.read_pickle('./Results/Feats_test_12_3yr.pkl')

train_feats_tot_36 = pd.read_pickle('./Results/Feats_train_36_3yr.pkl')
test_feats_tot_36 = pd.read_pickle('./Results/Feats_test_36_3yr.pkl')

In [18]:
train_feats_tot_12 = train_feats_tot_12[columns_12]
train_feats_tot_36 = train_feats_tot_36[columns_36]

test_feats_tot_12 = test_feats_tot_12[columns_12]
test_feats_tot_36 = test_feats_tot_36[columns_36]

In [19]:
#Change index to match the index of the predictions
train_feats_tot_12.index = [int(x+1) for x in train_feats_tot_12.index]
test_feats_tot_12.index = [int(x+1) for x in test_feats_tot_12.index]    

train_feats_tot_36.index = [int(x+1) for x in train_feats_tot_36.index]
test_feats_tot_36.index = [int(x+1) for x in test_feats_tot_36.index]    

ids = list(range(1, 8001))
ids = ['M{}'.format(x) for x in ids]

In [20]:
#Check which indices are both in the feats and the predictions
indices_used_12 = [x for x in list(train_feats_tot_12.index) if x in list(test_feats_tot_12.index)]
indices_used_12 = [x for x in indices_used_12 if x in preds_df_tot_test_12.index.get_level_values(0)]

indices_used_36 = [x for x in list(train_feats_tot_36.index) if x in list(test_feats_tot_36.index)]
indices_used_36 = [x for x in indices_used_36 if x in preds_df_tot_test_36.index.get_level_values(0)]

In [21]:
#Remove indices that are not both in the feats and the predictions
preds_df_tot_test_12 = preds_df_tot_test_12.loc[preds_df_tot_test_12.index.get_level_values(0).isin(indices_used_12)]
preds_df_tot_train_12 = preds_df_tot_train_12.loc[preds_df_tot_train_12.index.get_level_values(0).isin(indices_used_12)]

preds_df_tot_test_36 = preds_df_tot_test_36.loc[preds_df_tot_test_36.index.get_level_values(0).isin(indices_used_36)]
preds_df_tot_train_36 = preds_df_tot_train_36.loc[preds_df_tot_train_36.index.get_level_values(0).isin(indices_used_36)]

train_feats_tot_12 = train_feats_tot_12.loc[train_feats_tot_12.index.isin(indices_used_12)]
test_feats_tot_12 = test_feats_tot_12.loc[test_feats_tot_12.index.isin(indices_used_12)]

train_feats_tot_36 = train_feats_tot_36.loc[train_feats_tot_36.index.isin(indices_used_36)]
test_feats_tot_36 = test_feats_tot_36.loc[test_feats_tot_36.index.isin(indices_used_36)]

In [22]:
#Give the correct formatting for the FFORMA model
test_feats_tot_12.index.name = 'unique_id'
train_feats_tot_12.index.name = 'unique_id'

test_feats_tot_36.index.name = 'unique_id'
train_feats_tot_36.index.name = 'unique_id'


In [23]:
#Rematch indices

index_integers_12 = list(train_feats_tot_12.index)
index_integers_36 = list(train_feats_tot_36.index)


index_integers_multi_12 = np.repeat(index_integers_12, 12)
index_integers_multi_36 = np.repeat(index_integers_36, 36)

multi_index_12 = pd.MultiIndex.from_tuples(zip(index_integers_multi_12, list(preds_df_tot_train_12.index.get_level_values(1))), names = ['unique_id', 'ds'])
multi_index_36 = pd.MultiIndex.from_tuples(zip(index_integers_multi_36, list(preds_df_tot_train_36.index.get_level_values(1))), names = ['unique_id', 'ds'])

preds_df_tot_test_12.index = multi_index_12
preds_df_tot_train_12.index = multi_index_12

preds_df_tot_test_36.index = multi_index_36
preds_df_tot_train_36.index = multi_index_36

train_feats_tot_12.index = index_integers_12
train_feats_tot_12.index.name = 'unique_id'

test_feats_tot_36.index = index_integers_36
test_feats_tot_36.index.name = 'unique_id'

# Create errors

In [24]:
#Create errors dataframe
index_feats_12 = list(train_feats_tot_12.index)

index_feats_36 = list(train_feats_tot_36.index)

errors_df_tot_train_12 = pd.DataFrame(index = preds_df_tot_train_12.index, columns = preds_df_tot_train_12.columns)

errors_df_tot_train_36 = pd.DataFrame(index = preds_df_tot_train_36.index, columns = preds_df_tot_train_36.columns)

In [25]:
#Errors for training set

ts_val_actual = []
for i in index_feats_12:
    ts_val_actual.append(original_train_T_12['M{}'.format(i)].dropna().tolist()[-12:])

ts_val_actual_flat = [item for sublist in ts_val_actual for item in sublist]

for i in errors_df_tot_train_12.columns:
    errors_df_tot_train_12[i] = ts_val_actual_flat

#percentage errors
errors_df_tot_train_12 = ((errors_df_tot_train_12 - preds_df_tot_train_12)/errors_df_tot_train_12)*100

    
ts_val_actual = []
for i in index_feats_36:
    ts_val_actual.append(original_train_T_36['M{}'.format(i)].dropna().tolist()[-36:])

ts_val_actual_flat = [item for sublist in ts_val_actual for item in sublist]

for i in errors_df_tot_train_36.columns:
    errors_df_tot_train_36[i] = ts_val_actual_flat

#percentage errors
errors_df_tot_train_36 = ((errors_df_tot_train_36 - preds_df_tot_train_36)/errors_df_tot_train_36)*100


In [26]:
#Create dictionary that maps original index to new index
original_nrs_12 = list(set(list(errors_df_tot_train_12.index.get_level_values(0))))
original_nrs_dict_12 = dict(zip(original_nrs_12, list(range(len(original_nrs_12)))))
original_nrs_dict_rev_12 = {value:key for key, value in original_nrs_dict_12.items()}

original_nrs_36 = list(set(list(errors_df_tot_train_36.index.get_level_values(0))))
original_nrs_dict_36 = dict(zip(original_nrs_36, list(range(len(original_nrs_36)))))
original_nrs_dict_rev_36 = {value:key for key, value in original_nrs_dict_36.items()}

In [27]:
#Change index of errors df
errors_df_tot_train_12.index = pd.MultiIndex.from_tuples(zip(np.repeat(list(range(len(original_nrs_12))), 12), list(errors_df_tot_train_12.index.get_level_values(1))), names = ['unique_id', 'ds'])

errors_df_tot_train_36.index = pd.MultiIndex.from_tuples(zip(np.repeat(list(range(len(original_nrs_36))), 36), list(errors_df_tot_train_36.index.get_level_values(1))), names = ['unique_id', 'ds'])


In [28]:
train_feats_tot_12.index = list(range(len(original_nrs_12)))
train_feats_tot_12.index.name = 'unique_id'

train_feats_tot_36.index = list(range(len(original_nrs_36)))
train_feats_tot_36.index.name = 'unique_id'

In [29]:
test_feats_tot_12.index = list(range(len(original_nrs_12)))
test_feats_tot_12.index.name = 'unique_id'

test_feats_tot_36.index = list(range(len(original_nrs_36)))
test_feats_tot_36.index.name = 'unique_id'

In [30]:
preds_df_tot_test_12.index = pd.MultiIndex.from_tuples(zip(np.repeat(list(range(len(original_nrs_12))), 12), list(errors_df_tot_train_12.index.get_level_values(1))), names = ['unique_id', 'ds'])

preds_df_tot_test_36.index = pd.MultiIndex.from_tuples(zip(np.repeat(list(range(len(original_nrs_36))), 36), list(errors_df_tot_train_36.index.get_level_values(1))), names = ['unique_id', 'ds'])    

# Find bounds for CI

In [31]:
"""
Divide the dataset into two parts, A and B. Train the fforma on A and predict B, and vice versa.
You now have fforma predictions. 
Then train a meta-learning model that predicts the optimal ensemble
based on the MSIS measure""";

In [32]:
#Drop the LLevel
CI_df_train_12 = CI_df_train_12.drop(columns = 'LLevel')
CI_df_test_12 = CI_df_test_12.drop(columns = 'LLevel')

In [33]:
#Create a new preds dataframe with the correct column names
preds_df_tot_train_CI_12 = preds_df_tot_train_CI_12[CI_df_train_12.columns]
preds_df_tot_test_CI_12 = preds_df_tot_test_CI_12[CI_df_train_12.columns]

In [34]:
#Assuming that the bounds are symmetric, we keep only the lower bound
for i in CI_df_train_12.columns:
    CI_df_train_12[i] = CI_df_train_12[i].map(lambda x: x[-1])

for i in CI_df_test_12.columns:
    CI_df_test_12[i] = CI_df_test_12[i].map(lambda x: x[-1])    

In [35]:
#Keep only those indices that are in the prediction interval and in the prediction
CI_df_train_12 = CI_df_train_12.loc[CI_df_train_12.index.get_level_values(0).isin(preds_df_tot_train_CI_12.index.get_level_values(0))]
CI_df_test_12 = CI_df_test_12.loc[CI_df_test_12.index.get_level_values(0).isin(preds_df_tot_test_CI_12.index.get_level_values(0))]


In [36]:
#Take the differences to create the prediction interval 'radius'
differences_CI_train_12 = (CI_df_train_12 - preds_df_tot_train_CI_12)
differences_CI_test_12 = (CI_df_test_12 - preds_df_tot_test_CI_12)

In [37]:
all_ids_12 = list(set(errors_df_tot_train_12.index.get_level_values(0).unique()))

#Create set A and set B
y_1 = [x for x in all_ids_12 if x < 2000 or x > 6000]
y_2 = [x for x in all_ids_12 if x >= 2000 and x <= 6000]

train_feats_tot1_12 = train_feats_tot_12.loc[train_feats_tot_12.index.isin(y_1)]
train_feats_tot2_12 = train_feats_tot_12.loc[train_feats_tot_12.index.isin(y_2)]


In [38]:
#Create dictionaries to switch between old and new index
dict_index_part1 = dict(zip(y_1, range(len(y_1))))
dict_index_part2 = dict(zip(y_2, range(len(y_2))))

In [39]:
#Keep only correct errors
errors_df_tot_train1_12 = errors_df_tot_train_12.loc[errors_df_tot_train_12.index.get_level_values(0).isin(y_1)]
errors_df_tot_train2_12 = errors_df_tot_train_12.loc[errors_df_tot_train_12.index.get_level_values(0).isin(y_2)]

#Change index
errors_df_tot_train1_12.index = pd.MultiIndex.from_tuples(zip(np.repeat(range(len(y_1)), 12), list(range(1, 13))*len(y_1)), names=['unique_id', 'ds'])
errors_df_tot_train2_12.index = pd.MultiIndex.from_tuples(zip(np.repeat(range(len(y_2)), 12), list(range(1, 13))*len(y_2)), names=['unique_id', 'ds'])

#Get correct feats train
train_feats_tot1_12 = train_feats_tot_12.loc[train_feats_tot_12.index.isin(y_1)]
train_feats_tot2_12 = train_feats_tot_12.loc[train_feats_tot_12.index.isin(y_2)]

train_feats_tot1_12.index = range(len(y_1))
train_feats_tot2_12.index = range(len(y_2))
train_feats_tot1_12.index.name = 'unique_id'
train_feats_tot2_12.index.name = 'unique_id'

#Get correct feats test
test_feats_tot1_12 = test_feats_tot_12.loc[test_feats_tot_12.index.isin(y_1)]
test_feats_tot2_12 = test_feats_tot_12.loc[test_feats_tot_12.index.isin(y_2)]

test_feats_tot1_12.index = range(len(y_1))
test_feats_tot2_12.index = range(len(y_2))
test_feats_tot1_12.index.name = 'unique_id'
test_feats_tot2_12.index.name = 'unique_id'

In [40]:
train_feats_tot1_12 = train_feats_tot1_12[columns_12]
train_feats_tot2_12 = train_feats_tot2_12[columns_12]

In [41]:
if reload_CI:
    #Train the two models that determine the FFORMA forecasts
    optimal_params = {'n_estimators': 150,
                      'eta': 0.1,
                      'max_depth': 18,
                      'subsample': 0.8,
                      'colsample_bytree': 0.6}

    model1 = FFORMA(params=optimal_params, early_stopping_rounds = 10, verbose_eval=1, greedy_search = False)
    model2 = FFORMA(params=optimal_params, early_stopping_rounds = 10, verbose_eval=1, greedy_search = False)

    model1.fit(errors = errors_df_tot_train1_12, holdout_feats = train_feats_tot1_12, feats = test_feats_tot1_12)
    model2.fit(errors = errors_df_tot_train2_12, holdout_feats = train_feats_tot2_12, feats = test_feats_tot2_12)

    #Save the two models
    pickle.dump(model1.lgb, open("./Predictions/Final/Final/model1_new.pickle.dat", "wb"))
    pickle.dump(model2.lgb, open("./Predictions/Final/Final/model2_new.pickle.dat", "wb"))

# Train meta-learning models based on MAPE

In [42]:
drop_cols = []

if reload_meta_learn:
    optimal_params = {'n_estimators': 150,
                      'eta': 0.1,
                      'max_depth': 18,
                      'subsample': 0.8,
                      'colsample_bytree': 0.6}

    #Model for horizon 12
    model_12 = FFORMA(params=optimal_params, early_stopping_rounds = 10, verbose_eval=1, greedy_search = False, original_article = False)
    model_12.fit(errors=errors_df_tot_train_12.drop(columns=drop_cols), holdout_feats=train_feats_tot_12, feats=test_feats_tot_12)

    #Model for horizon 36
    model_36 = FFORMA(params=optimal_params, early_stopping_rounds = 10, verbose_eval=1, greedy_search = False, original_article = False)
    model_36.fit(errors=errors_df_tot_train_36.drop(columns=drop_cols), holdout_feats=train_feats_tot_36, feats=test_feats_tot_36)
    
    #Save models
    pickle.dump(model_12.lgb, open("./Models/model_12_150.pickle.dat", "wb"))
    pickle.dump(model_36.lgb, open("./Models/model_36_150.pickle.dat", "wb"))    
    

In [43]:
#Load all models
model_lgb = pickle.load(open('./Models/model.pickle.dat', 'rb'))
model_36_lgb = pickle.load(open('./Models/model_36_149.pickle.dat', 'rb'))

model1_lgb = pickle.load(open('./Models/model1_new.pickle.dat', 'rb'))
model2_lgb = pickle.load(open('./Models/model2_new.pickle.dat', 'rb'))

# Check FFORMA performance of MAPE in-sample

In [44]:
"""
12 mo - 5yr:
model_lgb = pickle.load(open('./Predictions/Final/Final/model_5yr_145.pickle.dat', 'rb'))
train_feats_tot_12 = pd.read_pickle('./Predictions/Final/Final/feats_train_12_5yr.pkl')
test_feats_tot_12 = pd.read_pickle('./Predictions/Final/Final/feats_test_12_5yr.pkl')

12 mo - 7yr:
model_lgb = pickle.load(open('./Predictions/Final/Final/model_7yr_137.pickle.dat', 'rb'))

36 mo - 3yr:
model_36_lgb = pickle.load(open('./Predictions/Final/Final/model_36_149.pickle.dat', 'rb'))

36 mo - 5yr:
""";

In [45]:
drop_cols=[]

#weights trainingset
weights_insample_12 = scipy.special.softmax(model_lgb.predict(train_feats_tot_12), axis = 1)
weights_insample_36 = scipy.special.softmax(model_36_lgb.predict(train_feats_tot_36), axis = 1)

In [46]:
#add fforma predictions to the models
preds_df_insample_12 = preds_df_tot_train_12.drop(columns=drop_cols).copy()
preds_df_insample_12['fforma'] = (np.repeat(weights_insample_12, 12, axis = 0)*preds_df_insample_12).sum(axis = 1)

preds_df_insample_36 = preds_df_tot_train_36.copy()
preds_df_insample_36['fforma'] = (np.repeat(weights_insample_36, 36, axis = 0)*preds_df_insample_36).sum(axis = 1)

In [47]:
#Insample fit:
errors_df_insample_12 = pd.DataFrame(index = preds_df_insample_12.index, columns = preds_df_insample_12.columns)

#Errors for train
ts_val_actual = []
for i in index_feats_12:
    ts_val_actual.append(original_train_T_12['M{}'.format(i)].dropna().tolist()[-12:])
ts_val_actual_flat = [item for sublist in ts_val_actual for item in sublist]
for i in errors_df_insample_12.columns:
    errors_df_insample_12[i] = ts_val_actual_flat    

In [48]:
#Insample fit:
errors_df_insample_36 = pd.DataFrame(index = preds_df_insample_36.index, columns = preds_df_insample_36.columns)

#Errors for train
ts_val_actual = []
for i in index_feats_36:
    ts_val_actual.append(original_train_T_36['M{}'.format(i)].dropna().tolist()[-36:])
ts_val_actual_flat = [item for sublist in ts_val_actual for item in sublist]
for i in errors_df_insample_36.columns:
    errors_df_insample_36[i] = ts_val_actual_flat       


In [49]:
#Turn into MAPE
errors_df_insample_12 = np.abs(((errors_df_insample_12 - preds_df_insample_12)/errors_df_insample_12)*100)
errors_df_insample_36 = np.abs(((errors_df_insample_36 - preds_df_insample_36)/errors_df_insample_36)*100)

errors_df_insample_12.index.names = ['unique_id', 'ds']
errors_df_insample_36.index.names = ['unique_id', 'ds']

errors_df_insample_12 = errors_df_insample_12.groupby('unique_id')[errors_df_insample_12.columns].mean()
errors_df_insample_36 = errors_df_insample_36.groupby('unique_id')[errors_df_insample_36.columns].mean()

In [50]:
#Horizon 12
errors_df_insample_12.describe().round(2).iloc[[1, 2, 3, 5, 7]]

,Naive,Naive2,SNaive,RWdrift,ETS,AutoArima,TBATS,DTrendS,RWDriftS,LLevelS,LLDTrends,fforma
mean,13.78,12.70,13.57,13.82,13.05,18.73,11.46,16.42,12.71,11.49,12.29,9.00
std,34.23,33.42,21.89,35.43,25.75,51.08,23.35,30.50,29.61,24.88,28.43,14.16
min,0.00,0.11,0.00,0.16,0.17,0.00,0.00,0.13,0.08,0.15,0.08,0.07
50%,6.92,5.70,7.13,6.89,6.24,7.14,5.43,6.97,5.71,5.55,5.60,4.61
max,1853.54,1853.54,555.41,1926.53,1374.58,2231.11,887.02,697.02,1761.91,1480.39,1671.49,556.64


In [51]:
#Horizon 36
errors_df_insample_36.describe().round(2).iloc[[1, 2, 3, 5, 7]]

,Naive,Naive2,SNaive,RWdrift,ETS,AutoArima,TBATS,DTrendS,RWDriftS,LLevelS,LLDTrends,fforma
mean,17.87,17.14,20.29,19.08,17.23,49.51,12174.25,27.29,20.40,16.67,19.98,13.75
std,29.63,29.59,32.25,33.99,29.01,98.23,1086675.65,61.00,38.08,28.65,38.99,20.53
min,0.00,0.19,0.17,0.12,0.00,0.14,0.00,0.25,0.28,0.11,0.28,0.26
50%,9.94,9.10,10.45,9.57,9.42,17.32,9.40,11.28,9.68,8.95,9.47,7.89
max,1150.05,1154.12,1135.19,1264.34,1115.50,2608.88,97177021.59,2342.72,1372.57,1145.15,1530.67,733.13


# Check performance FFORMA of MAPE out-of-sample

In [52]:
test_feats_tot_12 = test_feats_tot_12[columns_12]
test_feats_tot_36 = test_feats_tot_36[columns_36]

In [53]:
#weights testingset

weights_outsample_12 = scipy.special.softmax(model_lgb.predict(test_feats_tot_12), axis = 1)

weights_outsample_36 = scipy.special.softmax(model_36_lgb.predict(test_feats_tot_36), axis = 1)

In [54]:
#Check ensemble weights for horizon 12
pd.DataFrame(data = weights_outsample_12, columns = preds_df_tot_test_12.drop(columns=drop_cols).columns).describe().iloc[[1, 2, 3, 5, 7]].round(2)

,Naive,Naive2,SNaive,RWdrift,ETS,AutoArima,TBATS,DTrendS,RWDriftS,LLevelS,LLDTrends
mean,0.06,0.09,0.05,0.07,0.10,0.04,0.15,0.04,0.11,0.16,0.13
std,0.06,0.06,0.05,0.07,0.09,0.11,0.10,0.06,0.10,0.08,0.08
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,0.04,0.08,0.04,0.05,0.09,0.00,0.13,0.03,0.08,0.17,0.11
max,0.51,0.56,0.66,0.59,0.74,0.99,0.95,0.79,0.74,0.81,0.82


In [55]:
#Check ensemble weights for horizon 36
pd.DataFrame(data = weights_outsample_36, columns = preds_df_tot_test_36.drop(columns=drop_cols).columns).describe().iloc[[1, 2, 3, 5, 7]].round(2)

,Naive,Naive2,SNaive,RWdrift,ETS,AutoArima,TBATS,DTrendS,RWDriftS,LLevelS,LLDTrends
mean,0.10,0.13,0.07,0.09,0.15,0.01,0.12,0.02,0.04,0.21,0.06
std,0.08,0.06,0.06,0.08,0.07,0.04,0.13,0.03,0.04,0.07,0.07
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,0.10,0.13,0.06,0.06,0.15,0.00,0.08,0.01,0.03,0.22,0.04
max,0.76,0.72,0.70,0.68,0.70,0.84,0.92,0.30,0.47,0.59,0.82


In [56]:
#Create FFORMA forecasts
preds_df_outsample_12 = preds_df_tot_test_12.drop(columns=drop_cols).copy()
preds_df_outsample_12['fforma'] = (np.repeat(weights_outsample_12, 12, axis = 0)*preds_df_outsample_12).sum(axis = 1)

preds_df_outsample_36 = preds_df_tot_test_36.copy()
preds_df_outsample_36['fforma'] = (np.repeat(weights_outsample_36, 36, axis = 0)*preds_df_outsample_36).sum(axis = 1)

In [57]:
#Out-of-sample fit horizon 12
errors_df_outsample_12 = pd.DataFrame(index = preds_df_outsample_12.index, columns = preds_df_outsample_12.columns)

#Errors for train
ts_val_actual = []
for i in index_feats_12:
    ts_val_actual.append(original_test_T_12['M{}'.format(i)].dropna().tolist()[:12])

ts_val_actual_flat = [item for sublist in ts_val_actual for item in sublist]

for i in errors_df_outsample_12.columns:
    errors_df_outsample_12[i] = ts_val_actual_flat
    
errors_df_outsample_copy_12 = errors_df_outsample_12.copy()    

In [58]:
#Out-of-sample fit horizon 36
errors_df_outsample_36 = pd.DataFrame(index = preds_df_outsample_36.index, columns = preds_df_outsample_36.columns)

#Errors for train
ts_val_actual = []
for i in index_feats_36:
    cur_ts = original_train_T_36['M{}'.format(i)].dropna().tolist()[-24:] + original_test_T_36['M{}'.format(i)].dropna().tolist()[:12]
    ts_val_actual.append(cur_ts)
    
ts_val_actual_flat = [item for sublist in ts_val_actual for item in sublist]

for i in errors_df_outsample_36.columns:
    errors_df_outsample_36[i] = ts_val_actual_flat
    
errors_df_outsample_copy_36 = errors_df_outsample_36.copy()

In [59]:
#Transform into MAPE
errors_df_outsample_12 = np.abs(((errors_df_outsample_12 - preds_df_outsample_12)/errors_df_outsample_12)*100)
errors_df_outsample_12.index.names = ['unique_id', 'ds']
errors_df_outsample_12 = errors_df_outsample_12.groupby('unique_id')[errors_df_outsample_12.columns].mean()

errors_df_outsample_36 = np.abs(((errors_df_outsample_36 - preds_df_outsample_36)/errors_df_outsample_36)*100)
errors_df_outsample_36.index.names = ['unique_id', 'ds']
errors_df_outsample_36 = errors_df_outsample_36.groupby('unique_id')[errors_df_outsample_36.columns].mean()

In [60]:
#Check FFORMA performance out-of-sample horizon 12 months
errors_df_outsample_12.describe().round(2).iloc[[1, 2, 3, 5, 7]]

,Naive,Naive2,SNaive,RWdrift,ETS,AutoArima,TBATS,DTrendS,RWDriftS,LLevelS,LLDTrends,fforma
mean,14.19,12.99,13.92,14.11,12.66,18.69,11.55,16.13,13.21,11.44,12.59,10.61
std,25.92,23.39,20.17,26.44,21.76,44.19,20.64,27.46,26.59,20.85,24.45,18.34
min,0.00,0.15,0.00,0.15,0.11,0.00,0.00,0.12,0.09,0.11,0.09,0.09
50%,6.92,5.78,7.03,6.86,5.89,6.83,5.33,7.04,5.56,5.36,5.52,4.97
max,729.99,630.87,368.55,744.80,650.70,2083.55,656.14,419.87,794.15,690.63,794.15,405.16


In [61]:
#Check FFORMA performance out-of-sample horizon 36 months
errors_df_outsample_36.describe().round(2).iloc[[1, 2, 3, 5, 7]]

,Naive,Naive2,SNaive,RWdrift,ETS,AutoArima,TBATS,DTrendS,RWDriftS,LLevelS,LLDTrends,fforma
mean,17.18,16.37,18.43,18.32,16.87,42.54,21.15,25.11,19.87,15.94,19.49,15.21
std,23.95,23.37,25.77,26.55,23.53,99.59,94.40,50.63,31.69,22.36,31.01,23.61
min,0.00,0.32,0.26,0.40,0.00,0.18,0.00,0.20,0.23,0.28,0.23,0.22
50%,9.71,8.90,10.50,9.68,9.21,15.72,9.15,10.69,9.53,8.76,9.33,8.17
max,547.24,547.24,553.18,530.59,529.25,4763.18,5945.57,1765.89,712.37,509.89,681.55,914.96


# Train meta-learning models based on MSIS

In [62]:
#Create FFORMA predictions out-of-sample for part A and B
weights_outsample_y1_12 = scipy.special.softmax(model2_lgb.predict(train_feats_tot1_12), axis = 1)
weights_outsample_y2_12 = scipy.special.softmax(model1_lgb.predict(train_feats_tot2_12), axis = 1)

preds_df_insample_12 = preds_df_tot_train_12.copy()
preds_df_insample_12.index = preds_df_outsample_12.index

preds_df_insample1_12 = preds_df_insample_12.loc[preds_df_insample_12.index.get_level_values(0).isin(y_1)]
preds_df_insample2_12 = preds_df_insample_12.loc[preds_df_insample_12.index.get_level_values(0).isin(y_2)]

preds_df_insample1_12['fforma'] = (np.repeat(weights_outsample_y1_12, 12, axis = 0)*preds_df_insample1_12).sum(axis = 1)
preds_df_insample2_12['fforma'] = (np.repeat(weights_outsample_y2_12, 12, axis = 0)*preds_df_insample2_12).sum(axis = 1)

In [63]:
#Errors
errors_df_insample1_12 = pd.DataFrame(index = preds_df_insample1_12.index, columns = preds_df_insample1_12.columns)
errors_df_insample2_12 = pd.DataFrame(index = preds_df_insample2_12.index, columns = preds_df_insample2_12.columns)

y_1_original = [original_nrs_dict_rev_12[x] for x in y_1]
y_2_original = [original_nrs_dict_rev_12[x] for x in y_2]

ts_val_actual = []
for i in y_1_original:
        ts_val_actual.append(original_train_T_12['M{}'.format(i)].dropna().tolist()[-12:])
ts_val_actual_flat = [item for sublist in ts_val_actual for item in sublist]

for i in errors_df_insample1_12.columns:
    errors_df_insample1_12[i] = ts_val_actual_flat
    
ts_val_actual = []
for i in y_2_original:
        ts_val_actual.append(original_train_T_12['M{}'.format(i)].dropna().tolist()[-12:])
ts_val_actual_flat = [item for sublist in ts_val_actual for item in sublist]

for i in errors_df_insample2_12.columns:
    errors_df_insample2_12[i] = ts_val_actual_flat

In [64]:
#Duplicate the df
errors_df_insample1_copy_12 = errors_df_insample1_12.copy()
errors_df_insample2_copy_12 = errors_df_insample2_12.copy()

errors_df_insample1_12 = np.abs(((errors_df_insample1_12 - preds_df_insample1_12)/errors_df_insample1_12)*100)
errors_df_insample2_12 = np.abs(((errors_df_insample2_12 - preds_df_insample2_12)/errors_df_insample2_12)*100)

errors_df_insample1_12 = errors_df_insample1_12.groupby('unique_id')[errors_df_insample1_12.columns].mean()
errors_df_insample2_12 = errors_df_insample2_12.groupby('unique_id')[errors_df_insample2_12.columns].mean()


In [65]:
#Combine the FFORMA prediction dfs
preds_df_insample_CI_tot_12 = preds_df_insample1_12.append(preds_df_insample2_12)
preds_df_insample_CI_tot_12 = preds_df_insample_CI_tot_12.sort_index()

#Create a prediction interval df
CI_final_df_12 = pd.DataFrame(index = preds_df_insample_12.index, columns = CI_df_train_12.columns)

#put FFORMA predictions in all columns
for i in CI_final_df_12.columns:
    CI_final_df_12[i] = preds_df_insample_CI_tot_12['fforma'].tolist()

#Change index
differences_CI_train_vals_12 = list(differences_CI_train_12.index.get_level_values(0))
differences_CI_train_vals_12 = [original_nrs_dict_12[x] for x in differences_CI_train_vals_12]
differences_CI_train_12.index = pd.MultiIndex.from_tuples(zip(differences_CI_train_vals_12, differences_CI_train_12.index.get_level_values(1)), names = ['unique_id', 'ds'])

In [66]:
"""Function that determines the prediction interval"""
def get_interval(pd_series):
    Name = pd_series.name
    difference_series = differences_CI_train_12[Name]
    return [[x-y, x+y] for (x,y) in zip(list(pd_series), list(difference_series))]  

#Put the intervals in the df
for i in CI_final_df_12.columns:
    CI_final_df_12[i] = get_interval(CI_final_df_12[i])

#Combine the errors dfs
errors_df_insample_full_12 = errors_df_insample1_copy_12.append(errors_df_insample2_copy_12)
errors_df_insample_full_12 = errors_df_insample_full_12.sort_index()

#Drop columns
errors_df_insample_full_12 = errors_df_insample_full_12[CI_final_df_12.columns]

"""
Function that determines the MSIS value
"""
def get_MSIS_value(pd_series):
    Name = pd_series.name
    CI_series = list(CI_final_df_12[Name])
    
    #2 times the difference + 2/alpha (lower - actual) if actual<lower + 2/alpha (actual-higher) if actual>higher
    return [(y[1]-y[0]) +(2/0.05)*(y[0] - x) if x<y[0] else (y[1]-y[0]) +(2/0.05)*(x - y[1]) if x>y[1] else y[1]-y[0] for (x, y) in zip(list(pd_series), CI_series)]

#Calculate the MSIS values
for i in errors_df_insample_full_12.columns:
    errors_df_insample_full_12[i] = get_MSIS_value(errors_df_insample_full_12[i])

errors_df_insample_full_12 = errors_df_insample_full_12.groupby('unique_id').sum()
errors_df_insample_full_12 = errors_df_insample_full_12/12

#new index
index_real_12 = [original_nrs_dict_rev_12[x] for x in errors_df_insample_full_12.index]

In [67]:
#Calculate normalization for the MSIS measure (denominator of the MSIS equation)
index_seas_diff_12 = pd.MultiIndex.from_tuples(zip(np.repeat(errors_df_insample_full_12.index, 48), list(range(1, 49))*len(errors_df_insample_full_12.index)), names = ['unique_id', 'ds'])
errors_df_insample_full_seas_diff_12 = pd.DataFrame(index = index_seas_diff_12, columns = errors_df_insample_full_12.columns)

ts_val_actual = []
for i in index_real_12:
        ts_val_actual.append(original_train_T_12['M{}'.format(i)].dropna().tolist()[-12-48:-12])
ts_val_actual_flat = [item for sublist in ts_val_actual for item in sublist]

for i in errors_df_insample_full_seas_diff_12.columns:
    errors_df_insample_full_seas_diff_12[i] = ts_val_actual_flat

    
errors_df_insample_full_seas_diff_12 = errors_df_insample_full_seas_diff_12.groupby(level=0).diff(12).dropna()
errors_df_insample_full_seas_diff_12 = np.abs(errors_df_insample_full_seas_diff_12)
errors_df_insample_full_seas_diff_12 = errors_df_insample_full_seas_diff_12.groupby('unique_id').mean()

In [68]:
#Normalize the MSIS measure
errors_df_insample_full_12 = errors_df_insample_full_12/errors_df_insample_full_seas_diff_12

if reload_CI:
    optimal_params = {'n_estimators': 157,
                      'eta': 0.1,
                      'max_depth': 15,
                      'subsample': 0.8,
                      'colsample_bytree': 0.6}

    model_CI = FFORMA(params=optimal_params, early_stopping_rounds = 10, verbose_eval=1, greedy_search = False, CI=True)

    model_CI.fit(errors=errors_df_insample_full_12, holdout_feats=train_feats_tot_12, feats=test_feats_tot_12)
    
#Load the model
model_CI_12 = pickle.load(open('./Models/model_CI_12.pickle.dat', 'rb'))

# Check FFORMA performance of MSIS in-sample

In [69]:
#create the predictions
preds_insample_CI_12 = preds_df_insample_CI_tot_12[list(CI_df_train_12.columns) + ['fforma']]

#Get the ensemble weights for prediction interval
CI_pred_weights_insample_12 = scipy.special.softmax(model_CI_12.predict(train_feats_tot_12), axis = 1)


differences_CI_train_12['fforma'] = (np.repeat(CI_pred_weights_insample_12, 12, axis = 0)*differences_CI_train_12).sum(axis = 1)


#Now we need the actual values
actuals_insample_CI_12 = errors_df_insample1_copy_12.append(errors_df_insample2_copy_12)
actuals_insample_CI_12 = actuals_insample_CI_12.sort_index()
actuals_insample_CI_12 = actuals_insample_CI_12[differences_CI_train_12.columns]
actuals_insample_CI_copy_12 = actuals_insample_CI_12.copy()

In [70]:
def get_MSIS_value_final(pd_series):
    Name = pd_series.name
    
    differences = list(differences_CI_train_12[Name])
    actuals = list(pd_series)
    predictions = list(preds_insample_CI_12[Name])
    
    CI_series = list(differences_CI_train_12[Name])
    #x = differences, y = actuals, z = predictions    
    return [2*x + (2/0.05)*(z-x-y) if y<(z-x) else 2*x + (2/0.05)*(y-(z+x)) if y>(z+x) else 2*x for (x, y, z) in zip(differences, actuals, predictions)]

In [71]:
#Calculate final MSIS values and normalize
for i in actuals_insample_CI_12.columns:
    actuals_insample_CI_12[i] = get_MSIS_value_final(actuals_insample_CI_12[i])

actuals_insample_CI_12 = actuals_insample_CI_12.groupby('unique_id').mean()

errors_df_insample_full_seas_diff_12['fforma'] = errors_df_insample_full_seas_diff_12['AutoArima']

actuals_insample_CI_12 = actuals_insample_CI_12 / errors_df_insample_full_seas_diff_12

actuals_insample_CI_12.describe().round(2).iloc[[1, 2, 3, 5, 7]]

,AutoArima,TBATS,DTrendS,RWDriftS,LLevelS,LLDTrends,fforma
mean,15.02,9.77,10.59,7.70,7.69,7.44,6.23
std,24.48,19.10,20.34,15.22,17.04,16.08,14.67
min,0.04,0.18,0.15,0.08,0.76,0.08,0.30
50%,6.96,4.74,4.77,5.33,4.29,4.57,4.35
max,440.02,456.43,438.66,429.51,433.25,431.11,433.63


# Check FFORMA performance of MSIS out-of-sample

In [72]:
#Create weights
CI_pred_weights_outsample_12 = scipy.special.softmax(model_CI_12.predict(test_feats_tot_12), axis = 1)

#Create preds df
preds_df_outsample_CI_12 = preds_df_outsample_12[list(preds_df_tot_test_CI_12.columns) + ['fforma']]

#Create differences df
differences_CI_test_12.index = differences_CI_train_12.index
differences_CI_test_12['fforma'] = (np.repeat(CI_pred_weights_outsample_12, 12, axis = 0)*differences_CI_test_12).sum(axis = 1)

#Create errors df
errors_df_outsample_copy_fin_12 = errors_df_outsample_copy_12[differences_CI_test_12.columns]

In [73]:
def get_MSIS_value_final(pd_series):
    Name = pd_series.name
    
    differences = list(differences_CI_test_12[Name])
    actuals = list(pd_series)
    predictions = list(preds_df_outsample_CI_12[Name])
    
    #x = differences, y = actuals, z = predictions    
    return [2*x + (2/0.05)*(z-x-y) if y<(z-x) else 2*x + (2/0.05)*(y-(z+x)) if y>(z+x) else 2*x for (x, y, z) in zip(differences, actuals, predictions)]

In [74]:
#Create MSIS df and normalize
for i in errors_df_outsample_copy_fin_12.columns:
    errors_df_outsample_copy_fin_12[i] = get_MSIS_value_final(errors_df_outsample_copy_12[i])

errors_df_outsample_copy_fin_12 = errors_df_outsample_copy_fin_12.groupby('unique_id').mean()
errors_df_outsample_copy_fin_12 = errors_df_outsample_copy_fin_12 / errors_df_insample_full_seas_diff_12
errors_df_outsample_copy_fin_12.describe().round(2).iloc[[1, 2, 3, 5, 7]]

,AutoArima,TBATS,DTrendS,RWDriftS,LLevelS,LLDTrends,fforma
mean,15.96,10.28,12.37,8.74,8.71,8.56,7.31
std,40.09,28.93,33.20,30.04,30.92,30.43,26.98
min,0.08,0.13,0.26,0.21,0.34,0.21,0.33
50%,7.34,4.99,5.04,5.37,4.34,4.65,4.44
max,1826.66,1654.40,1734.59,1701.58,1732.05,1701.58,1680.60


# Check FFORMA performance of MSIS out-of-sample with FFORMA vs individual bounds

In [75]:
#Create preds df
preds_df_outsample_CI_final_12 = preds_df_outsample_CI_12.copy()

for i in preds_df_outsample_CI_final_12.columns:
    preds_df_outsample_CI_final_12[i] = preds_df_outsample_CI_final_12['fforma']

#For this dataframe, use FFORMA point predictions and statistical prediction intervals
def get_MSIS_value_final(pd_series):
    Name = pd_series.name
    
    differences = list(differences_CI_test_12[Name])
    actuals = list(pd_series)
    predictions = list(preds_df_outsample_CI_final_12[Name])
    
    #x = differences, y = actuals, z = predictions    
    return [2*x + (2/0.05)*(z-x-y) if y<(z-x) else 2*x + (2/0.05)*(y-(z+x)) if y>(z+x) else 2*x for (x, y, z) in zip(differences, actuals, predictions)]


errors_df_outsample_copy_finfin_12 = errors_df_outsample_copy_12[differences_CI_test_12.columns]

#Create MSIS 
for i in errors_df_outsample_copy_finfin_12.columns:
    errors_df_outsample_copy_finfin_12[i] = get_MSIS_value_final(errors_df_outsample_copy_12[i])

errors_df_outsample_copy_finfin_12 = errors_df_outsample_copy_finfin_12.groupby('unique_id').mean()
errors_df_outsample_copy_finfin_12 = errors_df_outsample_copy_finfin_12 / errors_df_insample_full_seas_diff_12
errors_df_outsample_copy_finfin_12.describe().round(2).iloc[[1, 2, 3, 5, 7]]

,AutoArima,TBATS,DTrendS,RWDriftS,LLevelS,LLDTrends,fforma
mean,9.13,9.11,8.64,7.82,7.53,7.66,7.31
std,27.18,27.95,28.76,27.02,27.71,27.45,26.98
min,0.11,0.13,0.19,0.21,0.34,0.21,0.33
50%,5.09,4.54,4.14,5.14,4.17,4.49,4.44
max,1673.08,1673.20,1674.31,1684.00,1685.12,1684.00,1680.60
